# Projet de scraping
<hr style='border-top: dashed 1px;'/>

## **scraping.py**

##### Le fichier *scraping.py* contient le code python permettant de récupérer les données sur le site de la CJUE. Attention, le code prend du temps à s'exécuter (entre 5 et 10 minutes) ; il est donc préférable de ne pas l'exécuter pour tester les autres fichiers (*dataframe.py* et *visualizer.py*), étant donné que les fichiers permetttant la création de la base de données sont déjà créés.<br>Exécuter le fichier scraping.py réinitialise les fichiers, il est donc préférable de ne pas l'exécuter si vous souhaitez garder les fichiers déjà créés.

Tout d'abord, on importe les librairies nécessaires au bon fonctionnement du code.

In [ ]:
from tqdm import tqdm  # Pour afficher la barre de progression
import requests  # Pour récupérer les pages web
from bs4 import BeautifulSoup  # Pour parser les pages web (i.e. les analyser)
import re  # Pour utiliser des expressions régulières (regex)

On a au préalable créé des fichiers .txt permettant de recueillir les données scrapées. On va donc créer une iste contenant les noms de ces fichiers et itérer sur cette liste pour vider le contenu des fichiers.

In [ ]:
# Création d'une liste de noms de fichiers préexistants à vider
noms_fichiers = ['affaires_dataframe.txt', 'cadres_dataframe.txt', 'cadres_raccourcis.txt', 'dates_dataframe.txt',
                 'documents_dataframe.txt', 'matieres_final.txt', 'liens_dataframe.txt', 'matieres.txt', 'matieres_raccourcis.txt', 'pays_dataframe.txt']

# Pour chaque nom de fichier dans la liste des noms de fichiers
for nom_fichier in noms_fichiers:
    with open(nom_fichier, 'r+') as fichier:  # Ouverture du fichier en mode lecture et écriture
        fichier.truncate(0)  # Troncature du fichier à 0 byte (i.e. suppression de son contenu)

On crée ensuite une liste contenant les liens des 9 pages internet de la CJUE qui nous intéressent dans le but de pouvoir itérer sur cette liste et récupérer le contenu de chacune de ces pages.

In [ ]:
# Récupération des liens des pages web contenant les documents de la CJUE et stockage dans une liste
urls = ['https://curia.europa.eu/juris/documents.jsf?lgrec=fr&nat=or&mat=FISC%252Cor&pcs=Oor&jur=C%2CT&td=%24mode%3DfromTo%24from%3D1952.01.01%24to%3D2023.03.04%3B%3B%3BPUB1%3BNPUB1%3B%3B%3BORDALL&lg=&dates=%2524type%253Dall%2524mode%253DfromTo%2524from%253D1954.01.01%2524to%253D2023.03.04&language=fr&pro=CONS%252COB%252C&cit=none%252CC%252CCJ%252CR%252C2008E%252C%252C%252C%252C%252C%252C%252C%252C%252C%252Ctrue%252Ctrue%252Ctrue&oqp=&page=' +
        str(page) + '&cid=756586' for page in range(1, 10)]

On crée ensuite une liste vide pour stocker les liens des documents pour ensuite itérer sur cette liste, accéder aux pages des documents et enfin appliquer des expressions régulières pour récupérer les informations qui nous intéressent.
On crée également une barre de progression pour suivre l'avancement du code concernant la récupération des informations sur les 9 pages de la CJUE filtrées auparavant.

In [ ]:
liens = []  # Liste vide pour stocker les liens des documents

barre_progression_urls = tqdm(
    total=len(urls), desc='Traitement des urls ', position=0, leave=True)  # Barre de progression pour le traitement des urls

Vient ensuite la boucle principale du code. On itère sur la liste des liens des pages de la CJUE et, pour chacune des pages, on récupère le contenu HTML de la page. On applique ensuite une expression régulière sur certaines parties du code HTML pour récupérer les informations qui nous intéressent et pour d'autres on peut directement récupérer la valeur de l'attribut de la balise HTML. On ajoute ensuite les informations récupérées dans les fichiers .txt créés au début du code afin de pouvoir les utiliser pour créer la base de données.

In [ ]:
for url in urls:  # Pour chaque URL dans la liste des URLs
    reponse = requests.get(url)  # Récupération de la page web

    # Utilisation de BeautifulSoup pour récupérer le contenu de la page web en html
    soup = BeautifulSoup(reponse.content, 'html.parser')

    # Trouver tous les élements td de la classe 'table_cell_nom_usuel' pour récupérer les noms des pays
    for td in soup.find_all('td', {'class': 'table_cell_nom_usuel'}):
        # Récupération du nom du pays
        pays = td.text.strip().split('/')[1].strip()

        # Suppression des parenthèses et du contenu entre parenthèses dans le nom du pays (ex: Royaume-Uni (Angleterre et Pays de Galles))
        pays = re.sub(' \([^)]+\)', '', pays)

        # Écriture du nom du pays dans le fichier pays_dataframe.txt
        # Ouverture du fichier pays_dataframe.txt en mode écriture et encodage en utf-8 pour les caractères spéciaux
        with open('pays_dataframe.txt', 'a', encoding='utf-8') as f:
            # Écriture du nom du pays dans le fichier pays_dataframe.txt sans le dernier caractère (espace)
            f.write(pays + '\n')

    # Trouver tous les élements td de la classe 'table_cell_date' pour récupérer les dates
    for td in soup.find_all('td', {'class': 'table_cell_date'}):
        date = td.text.strip()  # Récupération de la date

        # Écriture de la date dans le fichier dates_dataframe.txt
        # Ouverture du fichier dates_dataframe.txt en mode écriture
        with open('dates_dataframe.txt', 'a') as f:
            # Écriture de la date dans le fichier dates_dataframe.txt
            f.write(date + '\n')

    # Trouver tous les élements td de la classe 'table_cell_links_curia' pour récupérer les matières
    for td in soup.find_all('td', {'class': 'table_cell_links_curia'}):
        # Trouver l'élément div avec l'attribut id vide et récupérer le texte
        matiere = td.find('div', {'id': ''}).text.split()
        # Transformer la liste en chaîne de caractères
        matiere = ' '.join(matiere)

        # Expression régulière pour trouver les mots 'Fiscalité', 'Taxe sur la valeur ajoutée', 'Énergie', 'Libre circulation des marchandises - Restrictions quantitatives - Mesures d\'effet équivalent', 'Droits d\'accise' et 'Liberté d\'établissement'
        pattern1 = r'(?<!\S)(?:Fiscalité|Taxe sur la valeur ajoutée|Énergie|Libre circulation des marchandises - Restrictions quantitatives - Mesures d\'effet équivalent|Droits d\'accise|Liberté d\'établissement)(?!\S)'
        # Remplacer les mots par '- <mot>'
        matiere = re.sub(pattern1, r'- \g<0>', matiere)
        # Supprimer les mots 'Fiscalité - - '
        matiere = matiere.replace('Fiscalité - - ', '')

        # Expression régulière pour trouver les mots '- <mot> - Fiscalité'
        pattern2 = r'^-\s*([^-\n]*?)\s*-\s*Fiscalité\b'
        # Remplacer les mots par '- <mot>'
        matiere = re.sub(pattern2, r'- \1', matiere, flags=re.IGNORECASE)

        # Expression régulière pour trouver les mots '- <mot> - Énergie'
        pattern3 = r'^-\s*(?:.*-\s+)?([^-\n]*?)\s*-\s*Énergie\b'
        # Remplacer les mots par '- <mot>'
        matiere = re.sub(pattern3, r'- \1', matiere, flags=re.IGNORECASE)

        # Remplacer les mots '- - ' par '-'
        matiere = matiere.replace('- - ', '-')
        # Remplacer les mots ' - Fiscalité' par ' -'
        matiere = matiere.replace(' - Fiscalité', ' -')

        # Supprimer les mots ' -' en fin de chaîne et placer un '-' au début de la chaîne de caractères
        if matiere.endswith(' -'):
            matiere = '- ' + matiere[:-2].strip()

        # Supprimer les mots '- Fiscalité -', '- Fiscalité ', ' - ', ' -', '- ', '--' et les remplacer par '-'
        matiere = matiere.replace('- Fiscalité -', '-')
        matiere = matiere.replace('- Fiscalité ', '-')
        matiere = matiere.replace(' - ', '-')
        matiere = matiere.replace(' -', '-')
        matiere = matiere.replace('- ', '-')
        matiere = matiere.replace('--', '-')

        # Écriture de la matière dans le fichier matieres.txt
        # Ouverture du fichier matieres.txt en mode écriture
        with open('matieres.txt', 'a', encoding='utf-8') as f:
            # Écriture de la matière dans le fichier matieres.txt
            f.write(matiere + '\n')

    # Trouver tous les élements td de la classe 'table_cell_aff' pour récupérer les identifiants des affaires
    for td in soup.find_all('td', {'class': 'table_cell_aff'}):
        affaire = td.text.strip()  # Récupération de l'identifiant de l'affaire

        # Écriture de l'identifiant de l'affaire dans le fichier affaires_dataframe.txt
        # Ouverture du fichier affaires_dataframe.txt en mode écriture
        with open('affaires_dataframe.txt', 'a', encoding='utf-8') as f:
            # Écriture de l'identifiant de l'affaire dans le fichier affaires_dataframe.txt s'il n'est pas vide
            if affaire.strip():
                # Écriture de l'identifiant de l'affaire dans le fichier affaires_dataframe.txt
                f.write(affaire + '\n')

    # Trouver tous les élements td de la classe 'table_cell_doc' pour récupérer les identifiants des documents
    for td in soup.find_all('td', {'class': 'table_cell_doc'}):
        # Trouver l'élément span avec la classe 'outputEcli' et récupérer le texte
        ecli = td.find('span', {'class': 'outputEcli'}).text.strip()

        # Écriture de l'identifiant du document dans le fichier documents_dataframe.txt
        # Ouverture du fichier documents_dataframe.txt en mode écriture
        with open('documents_dataframe.txt', 'a', encoding='utf-8') as f:
            # Écriture de l'identifiant du document dans le fichier documents_dataframe.txt
            f.write(ecli + '\n')

    # Trouver tous les liens eur-lex de la page
    for lien in soup.find_all('a'):
        # Récupération de l'attribut href de l'élément a
        href = lien.get('href')

        if href and 'eur-lex' in href:  # Vérifier si l'attribut href n'est pas vide et contient 'eur-lex'
            # Ajout du lien eur-lex à la liste des liens eur-lex
            liens.append(href)

            # Écriture du lien eur-lex dans le fichier liens_dataframe.txt
            # Ouverture du fichier liens_dataframe.txt en mode écriture
            with open('liens_dataframe.txt', 'a') as f:
                # Écriture du lien eur-lex dans le fichier liens_dataframe.txt
                f.write(href + '\n')

    # Incrémenter la barre de progression des liens eur-lex
    barre_progression_urls.update(1)

barre_progression_urls.close()  # Fermer la barre de progression des liens eur-lex

On ferme la barre de progression pour le traitement des URLs des pages de la CJUE, on saute une ligne et on crée une nouvelle barre de progression pour le traitement des URLs des pages des documents. En effet, le traitement des 180 documents est l'opération la plus longue du code car on doit accéder à chacune des pages des documents, lire tout son contenu HTML et également rechercher des informations précises, notamment au moyen d'expressions régulières, ce qui prend entre 5 et 10 minutes, d'où l'importance de ne pas exécuter ce code si vous souhaitez garder les fichiers déjà créés. La barre de progression permet donc de suivre l'avancement du code et d'estimer le temps qu'il reste avant la fin du traitement.

In [ ]:
barre_progression_urls.close()  # Fermer la barre de progression des liens eur-lex

print()  # Saut de ligne entre les barres de progression

barre_progression_documents = tqdm(
    total=len(liens), desc='Traitement des documents ', position=0, leave=True)

L'opération suivante sur les documents est nécessaire car la catégorie "Matière" est souvent incomplète sur les pages de la CJUE ; il est donc essentiel d'accéder aux pages des documents pour récupérer plus de détails sur cette catégorie. On va donc itérer sur la liste des liens créée précédemment et accéder aux pages des documents.
On va appliquer un algorithme de recherche de chaîne de caractères pour récupérer plus de détails sur la catégorie "Matière".
En effet, on va rechercher dans le document *mots_cles.txt* si le contenu de chaque document contient une ou plusieurs chaîne de caractères présente dans ce fichier. Si c'est le cas, on ajoute cette chaîne de caractères au document .txt correspondant. On ferme ensuite la barre de progression pour le traitement des URLs des pages des documents.

In [ ]:
# Parcourir tous les liens de la liste des liens eur-lex pour récupérer les mots clés des documents
for lien in liens:
    response = requests.get(lien)  # Récupération de la page du document
    soup = BeautifulSoup(response.content, 'html.parser',
                         from_encoding='utf-8')  # Récupération du contenu de la page du document

    # Ouverture du fichier mots_cles.txt en mode lecture
    with open('mots_cles.txt', 'r', encoding='utf-8') as f:
        b = True  # Variable booléenne pour vérifier si le mot clé est trouvé ou non

        # Parcourir tous les mots clés du fichier mots_cles.txt
        for ligne in f:
            # Vérifier si le mot clé est trouvé dans le contenu du document
            if ligne.strip().lower() in str(soup).lower():
                # Écriture du mot clé dans le fichier cadres_dataframe.txt
                with open('cadres_dataframe.txt', 'a', encoding='utf-8') as f:
                    # Écriture du mot clé dans le fichier cadres_dataframe.txt
                    f.write(ligne.strip().replace('.', '') + ';')
                    b = False  # Modifier la valeur de la variable booléenne
        # Écriture de 'None' dans le fichier cadres_dataframe.txt si le mot clé n'est pas trouvé
        if b:
            # Ouverture du fichier cadres_dataframe.txt en mode écriture
            with open('cadres_dataframe.txt', 'a') as f:
                # Écriture de 'None' dans le fichier cadres_dataframe.txt
                f.write('None;')

        # Écriture d'un saut de ligne dans le fichier cadres_dataframe.txt
        with open('cadres_dataframe.txt', 'a') as f:
            # Écriture d'un saut de ligne dans le fichier cadres_dataframe.txt
            f.write('\n')

    # Incrémenter la barre de progression
    barre_progression_documents.update(1)

barre_progression_documents.close()  # Fermer la barre de progression

On va ensuite traiter le contenu du fichier *cadres_dataframe.txt* afin de pouvoir l'utiliser pour créer la base de données. On va donc ouvrir ce fichier, lire son contenu, puis créer un nouveau fichier *cadres_raccourcis.txt* qui contiendra le contenu du fichier *cadres_dataframe.txt* avec diverses modifications (voir ci-dessous).

In [ ]:
# Préparation des données pour la création de la dataframe

# Ouverture du fichier cadres_dataframe.txt en mode lecture
with open('cadres_dataframe.txt', 'r', encoding='utf-8') as f:
    contenu = f.read()  # Récupération du contenu du fichier cadres_dataframe.txt

    # Remplacer la phrase 'Taxe sur la valeur ajoutée' par 'TVA'
    contenu = contenu.replace('Taxe sur la valeur ajoutée', 'TVA')

    # Remplacer les mots 'Importés' et 'Importées' par 'Importation'
    contenu = contenu.replace('Importés', 'Importation')
    contenu = contenu.replace('Importées', 'Importation')

    # Remplacer les mots 'Exportés' et 'Exportées' par 'Exportation'
    contenu = contenu.replace('Exportés', 'Exportation')
    contenu = contenu.replace('Exportées', 'Exportation')

    # Remplacer les mots 'Accises' et 'Accise' par 'Droits d'accises'
    contenu = contenu.replace('Accises', 'Droits d\'accises')
    contenu = contenu.replace('Accise', 'Droits d\'accises')

    # Remplacer la phrase 'Produits énergétiques' par 'Taxation des produits énergétiques et de l’électricité'
    contenu = contenu.replace(
        'Produits énergétiques', 'Taxation des produits énergétiques et de l’électricité')

    # Remplacer le caractère "’" par "'"
    contenu = contenu.replace('’', "'")

    # Remplacer le mot 'Douanier' par 'Douane'
    contenu = contenu.replace('Douanier', 'Douane')

    # Remplacer la phrase 'Droits d'accise' par 'Droits d'accises'
    contenu = contenu.replace('Droits d\'accise', 'Droits d\'accises')

    # Remplacer la phrase 'Droits d'accisess' par 'Droits d'accises'
    contenu = contenu.replace('Droits d\'accisess', 'Droits d\'accises')

    # Remplacer la phrase 'Union douanière' par 'Douane'
    contenu = contenu.replace('Union douanière', 'Douane')

    # Remplacer la phrase 'Tarif douanier commun' par 'Douane'
    contenu = contenu.replace('Tarif douanier commun', 'Douane')

    # Remplacer la phrase 'Importation' par 'Douane'
    contenu = contenu.replace('Importation', 'Douane')

    # Remplacer la phrase 'Exportation' par 'Douane'
    contenu = contenu.replace('Exportation', 'Douane')

    # Remplacer la phrase 'Impositions intérieures' par 'Douane'
    contenu = contenu.replace('Impositions intérieures', 'Douane')


# Remplacement des mots dans le fichier cadres_raccourcis.txt
# Ouverture du fichier cadres_raccourcis.txt en mode écriture
with open('cadres_raccourcis.txt', 'w', encoding='utf-8') as f:
    # Écriture du contenu du fichier cadre_raccourcis.txt
    f.write(contenu)

On réitère la même opération de traitement sur le fichier *matieres.txt*, à savoir remplacer des chaînes de caractères comme "Taxe sur la valeur ajoutée" par "TVA" ou encore "Importés" et "Importées" par "Importation", le tout placé dans un nouveau fichier *matieres_raccourcis.txt*.

In [ ]:
# Ouverture du fichier matieres.txt en mode lecture
with open('matieres.txt', 'r', encoding='utf-8') as f:
    contenu = f.read()  # Récupération du contenu du fichier cadres_dataframe.txt

    # Remplacer la phrase 'Taxe sur la valeur ajoutée' par 'TVA'
    contenu = contenu.replace('Taxe sur la valeur ajoutée', 'TVA')

    # Remplacer les mots 'Importés' et 'Importées' par 'Importation'
    contenu = contenu.replace('Importés', 'Importation')
    contenu = contenu.replace('Importées', 'Importation')

    # Remplacer les mots 'Exportés' et 'Exportées' par 'Exportation'
    contenu = contenu.replace('Exportés', 'Exportation')
    contenu = contenu.replace('Exportées', 'Exportation')

    # Remplacer les mots 'Accises' et 'Accise' par 'Droits d'accises'
    contenu = contenu.replace('Accises', 'Droits d\'accises')
    contenu = contenu.replace('Accise', 'Droits d\'accises')

    # Remplacer la phrase 'Produits énergétiques' par 'Taxation des produits énergétiques et de l’électricité'
    contenu = contenu.replace(
        'Produits énergétiques', 'Taxation des produits énergétiques et de l’électricité')

    # Remplacer le mot 'Douanier' par 'Douane'
    contenu = contenu.replace('Douanier', 'Douane')

    # Remplacer la phrase 'Droits d'accise' par 'Droits d'accises'
    contenu = contenu.replace('Droits d\'accise', 'Droits d\'accises')

    # Remplacer le caractère "’" par "'"
    contenu = contenu.replace('’', "'")

    # Remplacer la phrase 'Droits d'accisess' par 'Droits d'accises'
    contenu = contenu.replace('Droits d\'accisess', 'Droits d\'accises')

    # Remplacer la phrase 'Union douanière' par 'Douane'
    contenu = contenu.replace('Union douanière', 'Douane')

    # Remplacer la phrase 'Tarif douanier commun' par 'Douane'
    contenu = contenu.replace('Tarif douanier commun', 'Douane')

    # Remplacer la phrase 'Importation' par 'Douane'
    contenu = contenu.replace('Importation', 'Douane')

    # Remplacer la phrase 'Exportation' par 'Douane'
    contenu = contenu.replace('Exportation', 'Douane')

    # Remplacer la phrase 'Impositions intérieures' par 'Douane'
    contenu = contenu.replace('Impositions intérieures', 'Douane')


# Remplacement des mots dans le fichier matieres_raccourcis.txt
# Ouverture du fichier matieres_raccourcis.txt en mode écriture
with open('matieres_raccourcis.txt', 'w', encoding='utf-8') as f:
    # Écriture du contenu du fichier matieres_raccourcis.txt
    f.write(contenu)

Une fois que l'on a les documents *cadres_raccourcis.txt* et *matieres_raccourcis.txt*, on peut dès lors créer un fichier définitif qui servira pour la création de la base de données : *matieres_final.txt*, qui contient les données relatives à la colonne "Matière". On ouvre donc le fichier *matieres_final.txt* en mode écriture et on itére sur chaque ligne des fichiers *cadres_raccourcis.txt* et *matieres_raccourcis.txt*.
Le fichier *matieres_final.txt* contiendra la ligne du fichier *matieres_raccourcis.txt* si elle ne contient pas la chaîne de caractères "-Fiscalité" (i.e. si la ligne du fichier *matieres_raccourcis.txt* contient un détail sur la catégorie "Matière", le but étant de ne pas garder le terme de "Fiscalité" sans précision).
Le cas échéant, on ajoute la ligne correspondante du fichier *cadres_raccourcis.txt* au fichier *matieres_final.txt*, ligne qui provient de l'itération sur le contenu du document correspondant à l'affaire (et non directement de la page de la CJUE).

In [ ]:
# Ouverture des fichiers matieres_raccourcis.txt et cadres_raccourcis.txt en mode lecture
with open('matieres_raccourcis.txt', 'r', encoding='utf-8') as matieres, open('cadres_raccourcis.txt', 'r', encoding='utf-8') as cadres:
    # Ouverture du fichier matieres_final.txt en mode écriture
    with open('matieres_final.txt', 'w', encoding='utf-8') as final:
        # Parcours des lignes du fichier matieres.txt et cadres_raccourcis.txt
        for ligne_matieres, ligne_cadres in zip(matieres, cadres):
            # Si la ligne contient '-Fiscalité', on prend la ligne correspondante dans cadres_raccourcis.txt
            if '-Fiscalité' in ligne_matieres:
                final.write('-' + ligne_cadres)
            else:
                # Sinon, on copie la ligne du fichier matieres.txt dans matieres_final.txt
                final.write(ligne_matieres)

***
## **dataframe.py**

##### Le fichier *dataframe.py* contient le code python permettant de créer la base de données à partir des fichiers créés par le fichier *scraping.py*. Il est donc nécessaire d'exécuter le fichier *scraping.py* avant d'exécuter le fichier *dataframe.py*. Le fichier *dataframe.py* sera par la suite utilisé dans le fichier *visualizer.py* pour répondre aux différentes questions posées.

Tout d'abord, on importe les librairies nécessaires au bon fonctionnement du code.

In [ ]:
import re  # Importation du module re pour les expressions régulières
import pandas as pd  # Importation du module pandas pour la création du dataframe

Dans ce fichier *dataframe.py*, on va préparer les données récoltées par le programme *scraping.py* pour créer la base de données par la suite. On va donc ouvrir le fichier *matieres_dataframe.txt*, créé au préalable, en mode lecture puis vider son contenu actuel. On crée ensuite une liste de doublons à retirer. En effet, en ayant déjà traité une certaine partie des informations dans le fichier *matieres_final.txt*, il reste à éliminer les doublons : par exemple une ligne contenant "TVA;Taxe sur la valeur ajoutée" avec "Taxe sur la valeur ajoutée" (le fichier *mots_cles.txt* contenant à la fois "TVA" et "Taxe sur la valeur ajoutée", il y a parfois des doublons) remplacée par "TVA" contiendra "TVA;TVA" après exécution du programme *scraping.py*, et il faut donc éliminer ce type de doublons.

In [ ]:
# Préparation de la donnée (matières) pour le dataframe
# Vider le contenu du fichier matieres_dataframe.txt
# Ouverture du fichier en mode lecture et écriture
with open('matieres_dataframe.txt', 'r+') as fichier:
    fichier.truncate(0)  # Vider le contenu du fichier

doublons = ['TVA', 'Droits d\'accises', 'Taxation des produits énergétiques et de l\'électricité', 'Importation'
            'Exportation'
            'Douane']  # Liste des mots à supprimer

On ouvre donc notre fichier *matieres_final.txt* en mode lecture et notre fichier *matieres_dataframe.txt* en mode écriture. On itère sur chaque ligne du fichier *matieres_final.txt* et on applique une expression régulière à chaque ligne pour séparer les différents mots (accollés par un point-virgule ou un tiret).
On crée une liste nommée "mots_uniques" pour chaque ligne du fichier *matieres_final.txt* et si la liste ne contient pas déjà un des mots de la ligne (en itérant sur les mots de la ligne courante), on l'ajoute à la liste.
On ajoute ensuite les mots de la liste "mots_uniques" dans la nouvelle ligne du fichier *matieres_dataframe.txt* et on peut ensuite commencer à créer la base de données, tous les fichiers étant complets, traités et prêts à être manipulés.

In [ ]:
# Ouverture des fichiers en mode lecture et écriture
with open('matieres_final.txt', 'r', encoding='utf-8') as f_entree, open('matieres_dataframe.txt', 'w', encoding='utf-8') as f_sortie:
    # Parcours du fichier d'entrée
    for ligne in f_entree:
        mots = re.split(r'[-;]', ligne)  # Séparation des mots
        mots_uniques = []  # Liste des mots uniques

        # Parcours des mots
        for mot in mots:
            if mot not in mots_uniques:  # Si le mot n'est pas déjà présent dans la liste des mots uniques
                # Ajout du mot dans la liste des mots uniques
                mots_uniques.append(mot)
        # Création de la nouvelle ligne
        new_line = '-'.join(mots_uniques)
        # Ecriture de la nouvelle ligne dans le fichier de sortie
        f_sortie.write(new_line)

Après avoir beaucoup travaillé sur la récupération des informations et leur traitement, toutes les données sont prêtes à l'emploi et on peut commencer à créer la base de données.
On commence donc par créer une liste contenant les noms des fichiers traités (terminant par "_dataframe.txt", hormis le fichier *cadres_dataframe.txt* qui était un fichier intermédiaire dans la récupération des informations relatives à la matière des affaires).
On crée ensuite une liste qui contiendra les différents dataframes (à partir des fichiers terminant par "_dataframe.txt") et on itère sur la liste contenant les noms de fichiers pour créer les dataframes associés.

In [ ]:
# Création du dataframe
# Création d'une liste contenant la liste des fichiers à lire
fichiers = ['affaires_dataframe.txt', 'dates_dataframe.txt',
            'documents_dataframe.txt', 'liens_dataframe.txt', 'pays_dataframe.txt']

dfs = []  # Liste des dataframes

# Parcours des fichiers
for fichier in fichiers:
    # Création du dataframe à partir du fichier et ajout dans la liste des dataframes
    df = pd.read_csv(fichier, sep=' ', names=[
                     fichier[:-14].capitalize()], encoding='utf-8')
    # Réinitialiser l'index du dataframe
    df.reset_index(drop=True, inplace=True)
    dfs.append(df)  # Ajout du dataframe dans la liste des dataframes

On crée ensuite le nom des colonnes pour chaque type de matière et on initialise leurs valeurs à 0 pour chaque affaire. Enfin, on concatène les dataframes pour créer le dataframe final.

In [ ]:
colonnes = ['TVA', 'Droits d\'accises', 'Taxation des produits énergétiques et de l\'électricité', 'Douane',
            'Droits de succession', 'Fusion', 'Taxe sur les véhicules', 'Libre circulation des capitaux',
            'Principes, objectifs et missions des Traités', 'Boissons spiritueuses', 'Libre circulation des marchandises',
            'Restrictions quantitatives', 'Mesures d\'effet équivalent', 'Liberté d\'établissement', 'Plus-values',
            'Coopération administrative', 'Tabacs manufacturés', 'Impôts indirects',
            'Citoyenneté de l\'Union', 'Libre circulation des travailleurs', 'Relations extérieures',
            'Association européenne de libre-échange', 'Taxe d\'immatriculation', 'Rapprochement des législations',
            'Adhésion', 'Dispositions financières', 'Ressources propres', 'Transports', 'Agriculture et Pêche',
            'Environnement', 'Départements français d\'outre-mer', 'Denrées alimentaires',
            'Monopoles d\'État à caractère commercial', 'Concurrence', 'Aides accordées par les États']

for colonne in colonnes:  # Parcours des colonnes
    # Création d'une nouvelle colonne dans le dataframe
    # Initialisation de la colonne à 0
    df[colonne] = [0 for i in range(len(df))]

df = pd.concat(dfs, axis=1)  # Concaténation des dataframes

On va maintenant remplir les valeurs des colonnes relatives aux différents types de matières. On va au préalable faire un dernier traitement sur les lignes du fichier *matieres_dataframe.txt* afin d'avoir un résultat plus synthétique (sinon on aurait beaucoup trop de colonnes dans le dataframe ; on peut donc par exemple placer "Alcool" et "Vin" dans la catégorie "Boissons spiritueuses" ou encore "Sucre" dans la catégorie "Denrées alimentaires").
On itère donc sur chaque ligne de fichier *matieres_dataframe.txt* et, en itérant sur chaque nom de colonne, on vérifie si le nom de la colonne est présent dans la ligne courante. Si c'est le cas, on remplace la valeur 0 par 1 dans la colonne correspondante.

In [ ]:
with open('matieres_dataframe.txt', 'r', encoding='utf-8') as fichier:
    lignes = fichier.readlines()  # Lecture des lignes du fichier

for i in range(len(df)):
    # Remplacement des doublons par des mots uniques dans la ligne i du fichier
    lignes[i].replace('Alcool', 'Boissons spiritueuses') \
        .replace('Tabac', 'Tabacs manufacturés') \
        .replace('Taxes sur l\'électricité', 'Taxation des produits énergétiques et de l\'électricité') \
        .replace('Taxes d\'effet équivalent', 'Mesures d\'effet équivalent') \
        .replace('Tarif douanier commun', 'Union douanière') \
        .replace('Sucre', 'Denrées alimentaires') \
        .replace('Vin', 'Boissons spiritueuses') \
        .replace('Fruits et légumes', 'Denrées alimentaires').replace('Douane-Douane', 'Douane').replace('Douane;Douane', 'Douane')

    # Boucle sur chaque colonne du dataframe
    for colonne in colonnes:
        # Si la colonne est présente dans la ligne i du fichier, on passe la valeur à 1
        if colonne in lignes[i]:
            df.loc[i, colonne] = 1

Pour finir, on crée une colonne "Total" afin de compter le nombre total de matières par affaire et, enfin, on exporte le dataframe final dans un fichier *dataframe.csv*.

In [ ]:
df['Total'] = df.iloc[:, 5:].sum(axis=1)  # Création de la colonne Total

# Enregistrement du dataframe dans un fichier csv
df.to_csv('dataframe.csv', index_label='Index')

***
## **visualizer.py**

##### Le fichier *visualizer.py* contient le code python permettant de répondre aux différentes questions posées. Il est donc nécessaire d'exécuter le fichier *dataframe.py* avant d'exécuter le fichier *visualizer.py*.

On importe la librairie **pandas**, nécessaire à la manipulation des données.

In [ ]:
import pandas as pd  # Importation du module pandas pour la création du dataframe

On crée alors une variable permettant de stocker les données du fichier *dataframe.csv* créé par le fichier *dataframe.py*. On crée ensuite un nouveau dataframe contenant seulement une colonne avec les différents pays (uniques) et une colonne "Total" qui contient le nombre total de condamnations par pays.
On crée enfin un nouveau dataframe contenant cette fois seulement les 5 pays ayant le plus de condamnations et on affiche le résultat.

In [ ]:
# Création du dataframe à partir du fichier dataframe.csv
df = pd.read_csv('dataframe.csv', sep=',', encoding='utf-8')

# Création d'un dataframe contenant la somme des totaux par pays
total_par_pays = df.groupby('Pays')['Total'].sum()
# Création d'un dataframe contenant les 5 pays ayant le plus de totaux
top_5_pays = total_par_pays.nlargest(5)

print(top_5_pays)  # Affichage du dataframe

On crée enfin un dictionnaire permettant de stocker les dataframes de chaque pays. On parcoure ensuite la liste des 5 pays ayant le plus de condamnations et on crée un dataframe pour chaque pays, contenant le nom du pays ainsi que les 5 matières dans lesquelles il a le plus de condamnations.
On affiche ensuite le dataframe de chacun des ces 5 pays.

In [ ]:
# Création d'un dictionnaire pour stocker les dataframes pour chaque pays
dataframes_par_pays = {}

# Création d'un dataframe pour chaque pays dans le dataframe top_5_pays avec les 5 matières les plus présentes
for pays in top_5_pays.index:  # Parcours des pays
    # Création d'un dataframe contenant les lignes du pays
    pays_df = df[df['Pays'] == pays]
    # Création d'une liste contenant les noms des colonnes des matières
    matieres = pays_df.columns[6:-1]
    # Création d'un dataframe contenant les 5 matières les plus présentes pour le pays
    top_matieres = pays_df[matieres].sum().nlargest(5)
    # Création d'un dataframe contenant les colonnes Pays et les 5 matières les plus présentes pour le pays
    pays_df = pays_df[['Pays'] + list(top_matieres.index)]
    # Réinitialisation des index du dataframe
    pays_df.reset_index(drop=True, inplace=True)
    # Aggrégation des lignes du dataframe pour avoir une seule ligne contenant le pays et le total de chaque matiere
    pays_df = pays_df.groupby('Pays')[list(top_matieres.index)].sum()
    # Ajout du dataframe dans le dictionnaire
    dataframes_par_pays[pays] = pays_df

# Afficher les dataFrames pour chaque pays dans le top 5
for pays, df in dataframes_par_pays.items():
    print(f"\nTop 5 matières pour {pays}")  # Affichage du nom du pays
    print(df)  # Affichage du dataframe